In [4]:
import requests
import csv
from datetime import datetime

# Fetch weather data from API
def fetch_weather(city: str, api_key: str) -> dict:
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric"
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()

        if response.status_code != 200:
            print(" API Error:", data.get("message", "Unknown error"))
            return {}

        return data

    except requests.exceptions.RequestException as e:
        print(" Network error:", e)
        return {}


# Analyze weather conditions
def analyze_weather(weather_data: dict) -> str:
    if not weather_data:
        return "No data to analyze."

    temp = weather_data["main"]["temp"]
    humidity = weather_data["main"]["humidity"]
    wind_speed = weather_data["wind"]["speed"]

    # Temperature conditions
    if temp <= 10:
        condition = "Cold (≤10°C)"
    elif 11 <= temp <= 24:
        condition = "Mild (11–24°C)"
    else:
        condition = "Hot (≥25°C)"

    alerts = []
    if wind_speed > 10:
        alerts.append("High wind alert!")
    if humidity > 80:
        alerts.append("Humid conditions!")

    if alerts:
        condition += " | " + " ".join(alerts)

    return condition


# Save weather data to CSV 
def save_to_csv(weather_data: dict, analysis: str, filename="weather_data.csv"):
    if not weather_data:
        print("No data to save.")
        return

    row = {
        "City": weather_data["name"],
        "Temperature (°C)": weather_data["main"]["temp"],
        "Humidity (%)": weather_data["main"]["humidity"],
        "Wind Speed (m/s)": weather_data["wind"]["speed"],
        "Weather Description": weather_data["weather"][0]["description"],
        "Analysis": analysis,
        "Date & Time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    try:
        with open(filename, "a", newline="", encoding="utf-8-sig") as file:
            writer = csv.DictWriter(file, fieldnames=row.keys())

            if file.tell() == 0:
                writer.writeheader()

            writer.writerow(row)

        print(f" Data saved to {filename}")

    except Exception as e:
        print(" CSV write error:", e)

# Main Program/ API_KEY function
if __name__ == "__main__":
    API_KEY = "65bef0f289c7946cfef78c98329fe368"
    city = input("Enter city name (city,country code): ").strip()

    weather_data = fetch_weather(city, API_KEY)
    analysis = analyze_weather(weather_data)

    print("\nWeather Analysis:")
    print(analysis)

    save_to_csv(weather_data, analysis)

Enter city name (city,country code):  bengaluru



Weather Analysis:
Mild (11–24°C)
 Data saved to weather_data.csv
